In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns

/usr/local/Anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [21]:
fopen = open('../../input/digit/train.csv', 'r')
train_df = pd.read_csv(fopen)
fopen.close()
fopen = open('../../input/digit/test.csv', 'r')
test_df = pd.read_csv(fopen)
fopen.close()

In [22]:
train_df.info()
print '----------------------'
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB
----------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [23]:
train_label = train_df.label.values
train_df.drop(['label'], inplace=True, axis=1)

train_arr = train_df.values.astype(np.float32) / 255.0
test_arr = test_df.values.astype(np.float32) / 255.0

In [24]:
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_arr, train_label, test_size=0.05, random_state=42)
n_sample = X_train.shape[0]
y_train = label_binarize(y_train, range(10))
y_test = label_binarize(y_test, range(10))

In [25]:
def add_fc_layer(input, input_size, output_size, activation=None):
    input = tf.reshape(input, [-1, input_size]) 
    w = tf.Variable(tf.truncated_normal([input_size, output_size], stddev=0.1, dtype=tf.float32))
    b = tf.Variable(tf.truncated_normal([1, output_size], stddev=0.1, dtype=tf.float32))
    out = tf.matmul(input, w) + b
    return activation(out) if activation else out

In [26]:
def add_conv_layer(x, filter_size, activation=None):
    w = tf.Variable(tf.truncated_normal(filter_size, stddev=0.1), dtype=tf.float32)
    conv = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME')
    b = tf.Variable(tf.truncated_normal([1, filter_size[-1]], stddev=0.1, dtype=tf.float32))
    out = conv + b
    return activation(out) if activation else out

def add_pool_layer(x, activation=None):
    pool = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    return activation(pool) if activation else pool

In [27]:
def acc(pred, target):
    return (np.argmax(pred, axis=1) == np.argmax(target, axis=1)).sum() / float(target.shape[0])

In [28]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
reshape = tf.reshape(x, [-1, 28, 28, 1])
keep_prob = tf.placeholder(tf.float32)

In [29]:
l1 = add_conv_layer(reshape, [5, 5, 1, 32], tf.nn.relu)
l2 = add_pool_layer(l1)

l3 = add_conv_layer(l2, [5, 5, 32, 64], tf.nn.relu)
l4 = add_pool_layer(l3)

l5 = add_fc_layer(l4, 7*7*64, 1024, tf.nn.relu)
drop = tf.nn.dropout(l5, keep_prob=keep_prob)
l6 = add_fc_layer(drop, 1024, 10)

soft_max = tf.nn.softmax(l6)

In [30]:
loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(l6, y))

In [31]:
opt = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [33]:
ss = tf.InteractiveSession()
ss.run(tf.initialize_all_variables())
for step in xrange(2000):
    start = step * 200 % X_train.shape[0]
    end = (step * 200 + 200) % X_train.shape[0]
    xs, ys = X_train[start:end], y_train[start:end]
    ss.run(opt, feed_dict={x : xs, y : ys, keep_prob : 0.5})
    if step % 50 == 0:
        train_result = ss.run(soft_max, feed_dict={x : xs, keep_prob : 1})
        test_result = ss.run(soft_max, feed_dict={x : X_test, keep_prob : 1})
        ls = ss.run(loss, feed_dict={x : xs, y : ys, keep_prob : 1})
        print "train_acc={0}, loss={1}, valid_acc={2}".format(acc(train_result, ys), ls, acc(test_result, y_test))
    

Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fe2d127bc10>> ignored


train_acc=0.15, loss=784.794799805, valid_acc=0.130952380952
train_acc=0.87, loss=104.557807922, valid_acc=0.83380952381
train_acc=0.895, loss=70.4955749512, valid_acc=0.902857142857
train_acc=0.91, loss=58.3571853638, valid_acc=0.922380952381
train_acc=0.895, loss=58.1383361816, valid_acc=0.932380952381
train_acc=0.935, loss=39.9743118286, valid_acc=0.94
train_acc=0.95, loss=30.6232070923, valid_acc=0.947142857143
train_acc=0.96, loss=27.3180236816, valid_acc=0.952857142857
train_acc=0.955, loss=34.3939628601, valid_acc=0.960476190476
train_acc=0.965, loss=26.5595169067, valid_acc=0.958571428571
train_acc=0.95, loss=22.8779373169, valid_acc=0.960952380952
train_acc=0.955, loss=21.6869277954, valid_acc=0.964285714286
train_acc=0.97, loss=23.3331546783, valid_acc=0.968095238095
train_acc=0.96, loss=28.4831047058, valid_acc=0.969523809524
train_acc=0.99, loss=11.6823406219, valid_acc=0.966666666667
train_acc=0.975, loss=18.5878601074, valid_acc=0.970476190476
train_acc=0.97, loss=20.1019

In [ ]:
res = ss.run(soft_max, feed_dict={x : test_arr, keep_prob : 1})
np.argmax(res, axis=1)

In [19]:
predict = np.array([])
for i in xrange(test_arr.shape[0] / 1000):
    res = ss.run(soft_max, feed_dict={x : test_arr[i * 1000: i * 1000 + 1000], keep_prob : 1})
    predict = np.append(predict, np.argmax(res, axis=1))

In [24]:
a = np.array([[1,2], [2, 3]])

array([[1, 2],
       [2, 3]])